In [1]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import os
import tqdm

class IDAO_validator:
    """
    Просто применяем TimeSeriesSplit отдельно для каждого из sat_id
    """
    
    def __init__(self, n_splits=5, max_train_size=None):
        self.n_splits = n_splits
        self.splitter = TimeSeriesSplit(n_splits, max_train_size=max_train_size)
        
    def get_n_splits(self):
        return self.n_splits
    
    def split(self, X):
        offset = 0
        for sat_id in X['sat_id'].unique():
            if sat_id != 0:
                offset += len(X[X['sat_id'] == (sat_id - 1)])
            for train_idx_, test_idx_ in self.splitter.split(X[X['sat_id'] == sat_id]):
                yield (train_idx_ + offset, test_idx_ + offset, sat_id)

In [2]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [3]:
# Загрузка данных
# windows
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

### Пример использования:

In [4]:
N_FOLDS = 5

idao_splitter = IDAO_validator(n_splits=N_FOLDS)

In [5]:
import lightgbm

In [6]:
%%time

## колонки, которые не нужны для предсказания
columns_to_drop = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'epoch', 'sat_id']

## целевые колонки
target_columns = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

## номер фолда для вывода логов
n_fold = 0
## предыдущий 'sat_id' - опять же для вывода логов - чтобы отслеживать смену объектов (sat_id)
prev_sat_id = 0

## список из значений smape (на тесте) для каждого из sat_id
smape_per_sat_id = []

for trn_idxs, tst_idxs, sat_id in idao_splitter.split(full_train):
    ## создаем массив для хранения предсказаний для теста, чтобы посчитать smape
    preds = np.zeros((tst_idxs.size, len(target_columns)))
    ## массив для хранения реальных значений целевых переменных для теста
    y_true = np.zeros((tst_idxs.size, len(target_columns)))
    
    ## для каждого sat_id обновляем n_fold (для логов)
    if prev_sat_id != sat_id:
            prev_sat_id = sat_id
            n_fold = 0
            print('-' * 15)
    n_fold += 1
    
    ## цикл по целевым колонкам
    for i, col in enumerate(target_columns):
        
        lgbm = lightgbm.LGBMRegressor(max_depth=5, learning_rate=0.2, n_estimators=1000, reg_alpha=5e-6,
                                      colsample_bytree = 0.85,
                                      reg_lambda=5e-4,
                                      random_state=13)
        X_train = full_train.iloc[trn_idxs].drop(columns=columns_to_drop).values
        y_train = full_train.iloc[trn_idxs][col].values

        X_test = full_train.iloc[tst_idxs].drop(columns=columns_to_drop).values
        y_test = full_train.iloc[tst_idxs][col].values

        lgbm.fit(X_train, y_train)
        
        preds[:, i] += lgbm.predict(X_test)
        y_true[:, i] = y_test
        
        ## выводим метрику для каждой целевой колонки отдельно
        print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: {col},\
              smape: {1 - smape(preds[:, i], y_test):.5f}')
    ## выводим метрику для всех целевых колонок сразу (для одного sat_id)
    print('_' * 15)
    print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: ALL_COLUMNS,\
              smape: {1 - smape(preds, y_true):.5f}')
    smape_per_sat_id.append(smape(preds, y_true))
    print('_' * 15)

sat_id: 0, fold: 1, target_col: x,              smape: 0.90240
sat_id: 0, fold: 1, target_col: y,              smape: 0.91588
sat_id: 0, fold: 1, target_col: z,              smape: 0.90315
sat_id: 0, fold: 1, target_col: Vx,              smape: 0.91755
sat_id: 0, fold: 1, target_col: Vy,              smape: 0.88664
sat_id: 0, fold: 1, target_col: Vz,              smape: 0.91993
_______________
sat_id: 0, fold: 1, target_col: ALL_COLUMNS,              smape: 0.90759
_______________
sat_id: 0, fold: 2, target_col: x,              smape: 0.96533
sat_id: 0, fold: 2, target_col: y,              smape: 0.97928
sat_id: 0, fold: 2, target_col: z,              smape: 0.96584
sat_id: 0, fold: 2, target_col: Vx,              smape: 0.97736
sat_id: 0, fold: 2, target_col: Vy,              smape: 0.93304
sat_id: 0, fold: 2, target_col: Vz,              smape: 0.97962
_______________
sat_id: 0, fold: 2, target_col: ALL_COLUMNS,              smape: 0.96675
_______________
sat_id: 0, fold: 3, target_c

sat_id: 3, fold: 3, target_col: x,              smape: 0.94227
sat_id: 3, fold: 3, target_col: y,              smape: 0.97856
sat_id: 3, fold: 3, target_col: z,              smape: 0.90156
sat_id: 3, fold: 3, target_col: Vx,              smape: 0.93522
sat_id: 3, fold: 3, target_col: Vy,              smape: 0.88324
sat_id: 3, fold: 3, target_col: Vz,              smape: 0.92476
_______________
sat_id: 3, fold: 3, target_col: ALL_COLUMNS,              smape: 0.92760
_______________
sat_id: 3, fold: 4, target_col: x,              smape: 0.95053
sat_id: 3, fold: 4, target_col: y,              smape: 0.98270
sat_id: 3, fold: 4, target_col: z,              smape: 0.96265
sat_id: 3, fold: 4, target_col: Vx,              smape: 0.98211
sat_id: 3, fold: 4, target_col: Vy,              smape: 0.95975
sat_id: 3, fold: 4, target_col: Vz,              smape: 0.96753
_______________
sat_id: 3, fold: 4, target_col: ALL_COLUMNS,              smape: 0.96754
_______________
sat_id: 3, fold: 5, target_c

sat_id: 6, fold: 5, target_col: x,              smape: 0.86805
sat_id: 6, fold: 5, target_col: y,              smape: 0.87168
sat_id: 6, fold: 5, target_col: z,              smape: 0.87992
sat_id: 6, fold: 5, target_col: Vx,              smape: 0.86541
sat_id: 6, fold: 5, target_col: Vy,              smape: 0.88189
sat_id: 6, fold: 5, target_col: Vz,              smape: 0.88536
_______________
sat_id: 6, fold: 5, target_col: ALL_COLUMNS,              smape: 0.87538
_______________
---------------
sat_id: 7, fold: 1, target_col: x,              smape: 0.56312
sat_id: 7, fold: 1, target_col: y,              smape: 0.56825
sat_id: 7, fold: 1, target_col: z,              smape: 0.53557
sat_id: 7, fold: 1, target_col: Vx,              smape: 0.26620
sat_id: 7, fold: 1, target_col: Vy,              smape: 0.11220
sat_id: 7, fold: 1, target_col: Vz,              smape: 0.13973
_______________
sat_id: 7, fold: 1, target_col: ALL_COLUMNS,              smape: 0.36418
_______________
sat_id: 7, f

sat_id: 10, fold: 2, target_col: x,              smape: 0.98841
sat_id: 10, fold: 2, target_col: y,              smape: 0.95594
sat_id: 10, fold: 2, target_col: z,              smape: 0.96019
sat_id: 10, fold: 2, target_col: Vx,              smape: 0.94964
sat_id: 10, fold: 2, target_col: Vy,              smape: 0.97533
sat_id: 10, fold: 2, target_col: Vz,              smape: 0.98092
_______________
sat_id: 10, fold: 2, target_col: ALL_COLUMNS,              smape: 0.96840
_______________
sat_id: 10, fold: 3, target_col: x,              smape: 0.95535
sat_id: 10, fold: 3, target_col: y,              smape: 0.96160
sat_id: 10, fold: 3, target_col: z,              smape: 0.95902
sat_id: 10, fold: 3, target_col: Vx,              smape: 0.92380
sat_id: 10, fold: 3, target_col: Vy,              smape: 0.96194
sat_id: 10, fold: 3, target_col: Vz,              smape: 0.93621
_______________
sat_id: 10, fold: 3, target_col: ALL_COLUMNS,              smape: 0.94965
_______________
sat_id: 10, fo

sat_id: 13, fold: 3, target_col: Vy,              smape: 0.89481
sat_id: 13, fold: 3, target_col: Vz,              smape: 0.87555
_______________
sat_id: 13, fold: 3, target_col: ALL_COLUMNS,              smape: 0.89382
_______________
sat_id: 13, fold: 4, target_col: x,              smape: 0.97332
sat_id: 13, fold: 4, target_col: y,              smape: 0.93140
sat_id: 13, fold: 4, target_col: z,              smape: 0.95607
sat_id: 13, fold: 4, target_col: Vx,              smape: 0.96211
sat_id: 13, fold: 4, target_col: Vy,              smape: 0.93098
sat_id: 13, fold: 4, target_col: Vz,              smape: 0.95837
_______________
sat_id: 13, fold: 4, target_col: ALL_COLUMNS,              smape: 0.95204
_______________
sat_id: 13, fold: 5, target_col: x,              smape: 0.94322
sat_id: 13, fold: 5, target_col: y,              smape: 0.93795
sat_id: 13, fold: 5, target_col: z,              smape: 0.94205
sat_id: 13, fold: 5, target_col: Vx,              smape: 0.95087
sat_id: 13, fo

...

sat_id: 584, fold: 2, target_col: Vy,              smape: 0.91828
sat_id: 584, fold: 2, target_col: Vz,              smape: 0.96990
_______________
sat_id: 584, fold: 2, target_col: ALL_COLUMNS,              smape: 0.94363
_______________
sat_id: 584, fold: 3, target_col: x,              smape: 0.93683
sat_id: 584, fold: 3, target_col: y,              smape: 0.93697
sat_id: 584, fold: 3, target_col: z,              smape: 0.95454
sat_id: 584, fold: 3, target_col: Vx,              smape: 0.93597
sat_id: 584, fold: 3, target_col: Vy,              smape: 0.92677
sat_id: 584, fold: 3, target_col: Vz,              smape: 0.95777
_______________
sat_id: 584, fold: 3, target_col: ALL_COLUMNS,              smape: 0.94148
_______________
sat_id: 584, fold: 4, target_col: x,              smape: 0.91388
sat_id: 584, fold: 4, target_col: y,              smape: 0.96221
sat_id: 584, fold: 4, target_col: z,              smape: 0.93653
sat_id: 584, fold: 4, target_col: Vx,              smape: 0.91196


sat_id: 587, fold: 4, target_col: x,              smape: 0.43877
sat_id: 587, fold: 4, target_col: y,              smape: 0.44387
sat_id: 587, fold: 4, target_col: z,              smape: 0.44182
sat_id: 587, fold: 4, target_col: Vx,              smape: 0.42494
sat_id: 587, fold: 4, target_col: Vy,              smape: 0.39365
sat_id: 587, fold: 4, target_col: Vz,              smape: 0.44975
_______________
sat_id: 587, fold: 4, target_col: ALL_COLUMNS,              smape: 0.43213
_______________
sat_id: 587, fold: 5, target_col: x,              smape: 0.44718
sat_id: 587, fold: 5, target_col: y,              smape: 0.38604
sat_id: 587, fold: 5, target_col: z,              smape: 0.38232
sat_id: 587, fold: 5, target_col: Vx,              smape: 0.38722
sat_id: 587, fold: 5, target_col: Vy,              smape: 0.39921
sat_id: 587, fold: 5, target_col: Vz,              smape: 0.38708
_______________
sat_id: 587, fold: 5, target_col: ALL_COLUMNS,              smape: 0.39817
_______________


sat_id: 590, fold: 5, target_col: Vx,              smape: 0.78934
sat_id: 590, fold: 5, target_col: Vy,              smape: 0.84905
sat_id: 590, fold: 5, target_col: Vz,              smape: 0.81523
_______________
sat_id: 590, fold: 5, target_col: ALL_COLUMNS,              smape: 0.81546
_______________
---------------
sat_id: 591, fold: 1, target_col: x,              smape: 0.96641
sat_id: 591, fold: 1, target_col: y,              smape: 0.90648
sat_id: 591, fold: 1, target_col: z,              smape: 0.97368
sat_id: 591, fold: 1, target_col: Vx,              smape: 0.87134
sat_id: 591, fold: 1, target_col: Vy,              smape: 0.95937
sat_id: 591, fold: 1, target_col: Vz,              smape: 0.90904
_______________
sat_id: 591, fold: 1, target_col: ALL_COLUMNS,              smape: 0.93105
_______________
sat_id: 591, fold: 2, target_col: x,              smape: 0.98269
sat_id: 591, fold: 2, target_col: y,              smape: 0.98263
sat_id: 591, fold: 2, target_col: z,             

sat_id: 594, fold: 1, target_col: Vz,              smape: 0.97844
_______________
sat_id: 594, fold: 1, target_col: ALL_COLUMNS,              smape: 0.97399
_______________
sat_id: 594, fold: 2, target_col: x,              smape: 0.95879
sat_id: 594, fold: 2, target_col: y,              smape: 0.98242
sat_id: 594, fold: 2, target_col: z,              smape: 0.97626
sat_id: 594, fold: 2, target_col: Vx,              smape: 0.97903
sat_id: 594, fold: 2, target_col: Vy,              smape: 0.98082
sat_id: 594, fold: 2, target_col: Vz,              smape: 0.96030
_______________
sat_id: 594, fold: 2, target_col: ALL_COLUMNS,              smape: 0.97294
_______________
sat_id: 594, fold: 3, target_col: x,              smape: 0.97370
sat_id: 594, fold: 3, target_col: y,              smape: 0.98649
sat_id: 594, fold: 3, target_col: z,              smape: 0.98178
sat_id: 594, fold: 3, target_col: Vx,              smape: 0.95856
sat_id: 594, fold: 3, target_col: Vy,              smape: 0.95914


sat_id: 597, fold: 3, target_col: x,              smape: 0.82107
sat_id: 597, fold: 3, target_col: y,              smape: 0.86071
sat_id: 597, fold: 3, target_col: z,              smape: 0.83335
sat_id: 597, fold: 3, target_col: Vx,              smape: 0.86919
sat_id: 597, fold: 3, target_col: Vy,              smape: 0.81453
sat_id: 597, fold: 3, target_col: Vz,              smape: 0.85346
_______________
sat_id: 597, fold: 3, target_col: ALL_COLUMNS,              smape: 0.84205
_______________
sat_id: 597, fold: 4, target_col: x,              smape: 0.93875
sat_id: 597, fold: 4, target_col: y,              smape: 0.95090
sat_id: 597, fold: 4, target_col: z,              smape: 0.87472
sat_id: 597, fold: 4, target_col: Vx,              smape: 0.92264
sat_id: 597, fold: 4, target_col: Vy,              smape: 0.90926
sat_id: 597, fold: 4, target_col: Vz,              smape: 0.92874
_______________
sat_id: 597, fold: 4, target_col: ALL_COLUMNS,              smape: 0.92083
_______________


In [7]:
print(1 - np.mean(smape_per_sat_id))

0.9029799649135525


In [13]:
0.9029799649135525

0.9029799649135525

In [9]:
for col_name in target_columns:
    full_test[col_name] = 0.0

In [10]:
for sat_id in tqdm.tqdm_notebook(full_test['sat_id'].unique()):

    ## цикл по целевым колонкам
    for i, col in enumerate(target_columns):

        lgbm = lightgbm.LGBMRegressor(max_depth=5, learning_rate=0.2, n_estimators=1000, reg_alpha=5e-6,
                                      colsample_bytree = 0.85,
                                      reg_lambda=5e-4,
                                      random_state=13)
        X_train = full_train[full_train['sat_id'] == sat_id].drop(columns=columns_to_drop).values
        y_train = full_train[full_train['sat_id'] == sat_id][col].values

        X_test = full_test[full_test['sat_id'] == sat_id].drop(columns=columns_to_drop).values
        y_test = full_test[full_test['sat_id'] == sat_id][col].values

        lgbm.fit(X_train, y_train)

        full_test.loc[full_test['sat_id'] == sat_id, col] = lgbm.predict(X_test)

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [11]:
full_test[['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('submission.csv', index_label='id')

In [12]:
full_test.head()

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x,y,z,Vx,Vy,Vz
id,,,,,,,,,,,,,,
3927,1,2014-02-01T00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774,-24217.007562,-11256.393631,6646.571927,2.716798,-1.242569,1.042418
3928,1,2014-02-01T00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402,-20358.965098,-12423.973084,7822.231772,3.352913,-0.857543,0.844714
3929,1,2014-02-01T00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412,-15643.936446,-13477.645508,8864.152417,4.089137,-0.388634,0.518531
3930,1,2014-02-01T01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600,-8801.671951,-13253.673766,9110.181645,4.887448,0.572101,-0.071621
3931,1,2014-02-01T01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724,-4742.436952,-12707.648175,8935.703646,5.271242,1.315763,-0.662347
